**Autor** : Kauvin Lucas

**e-mail**: 

#  <a name="resumo"> RESUMO </a>

Este notebook tem o objetivo de mostrar como criamos nossos módulos customizados e o uso deles nos códigos

#  <a name="indice">  Índice </a>

* [Resumo](#resumo)
* [Índice](#indice)
* [Questão 1 - Crie uma função que encontra filtra os números ímpares de uma lista de inteiros e depois os multiplica por 3.](#secao_1)
* [Questão 2 - Crie um módulo que contenha essa função e depois faça o import.](#secao_2)
* [Questão 3 - Crie testes unitários para as funções.](#secao_3)
* [Questão 4 - Faça com que o módulo gere logs e registre em arquivo.](#secao_4)
* [Questão 5 - Use Decorators para incluir mensagens de logs nas funções criadas.](#secao_5)
* [Conclusão](#conclusao)

#  <a name="secao_1"> Questão 1 - Crie uma função que encontra filtra os números ímpares de uma lista de inteiros e depois os multiplica por 3.</a>
[Voltar ao índice](#indice)

In [1]:
from numpy.random import randint
from importlib import reload

In [2]:
inteiros_15 = list(randint(10,size=15))

In [3]:
inteiros_15

[3, 0, 0, 4, 4, 6, 7, 4, 2, 4, 2, 3, 8, 7, 3]

In [12]:
def filtr_n_mult(lista, factor = 3):
    impares = [a for a in lista if a%2 !=0]
    por_tres = [n*factor for n in impares]
    return por_tres

In [13]:
filtr_n_mult(inteiros_15, factor=3)

[9, 21, 9, 21, 9]

#  <a name="secao_2"> Questão 2 - Crie um módulo que contenha essa função e depois faça o import.</a>
[Voltar ao índice](#indice)

In [14]:
from main_module import filtr_n_mult as fm

In [15]:
dir(fm)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [16]:
fm(inteiros_15)

[9, 21, 9, 21, 9]

#  <a name="secao_3"> Questão 3 - Crie testes unitários para as funções. </a>
[Voltar ao índice](#indice)

Ref: https://docs.python.org/pt-br/3/howto/logging.html

In [25]:
! python -m unittest test_main_module.Filtr_n_Mult

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


#  <a name="secao_4"> Questão 4 - Faça com que o módulo gere logs e registre em arquivo.</a>
[Voltar ao índice](#indice)

In [60]:
import main_module as fm

In [64]:
import logging
logging.basicConfig()

In [74]:
# Mostrar logs Error (sem erro)
logging.getLogger().setLevel(logging.ERROR)
fm.filtr_n_mult(inteiros_15)

[9, 21, 9, 21, 9]

In [75]:
# Mostrar logs Error (com erro)
logging.getLogger().setLevel(logging.ERROR)
fm.filtr_n_mult(['as']) # Erro

ERROR:main_module:Error executing the function


In [57]:
# Mostrar logs de Warning
logging.getLogger().setLevel(logging.WARNING)
fm.filtr_n_mult(inteiros_15)

[9, 21, 9, 21, 9]

In [56]:
# Mostrar logs de info e Warning
logging.getLogger().setLevel(logging.INFO)
fm.filtr_n_mult(inteiros_15)

INFO:main_module:Function has completed executing


[9, 21, 9, 21, 9]

In [58]:
# Mostrar logs de Info, Warning e Debug
logging.getLogger().setLevel(logging.DEBUG)
fm.filtr_n_mult(inteiros_15)

INFO:main_module:Function has completed executing
DEBUG:main_module:Factor is set to 3 by default


[9, 21, 9, 21, 9]

#  <a name="secao_5"> Questão 5 - Use Decorators para incluir mensagens de logs nas funções criadas. </a>
[Voltar ao índice](#indice)

Ref: https://dev.to/aldo/implementing-logging-in-python-via-decorators-1gje

In [76]:
import logging
import functools


def _generate_log(path):
    """
    Create a logger object
    :param path: Path of the log file.
    :return: Logger object.
    """
    # Create a logger and set the level.
    logger = logging.getLogger('LogError')
    logger.setLevel(logging.ERROR)

    # Create file handler, log format and add the format to file handler
    file_handler = logging.FileHandler(path)

    # See https://docs.python.org/3/library/logging.html#logrecord-attributes
    # for log format attributes.
    log_format = '%(levelname)s %(asctime)s %(message)s'
    formatter = logging.Formatter(log_format)
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    return logger


def log_error(path='log.error.log'):
    """
    We create a parent function to take arguments
    :param path:
    :return:
    """

    def error_log(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):

            try:
                # Execute the called function, in this case `divide()`.
                # If it throws an error `Exception` will be called.
                # Otherwise it will be execute successfully.
                return func(*args, **kwargs)
            except Exception as e:
                logger = _generate_log(path)
                error_msg = 'And error has occurred at /' + func.__name__ + '\n'
                logger.exception(error_msg)

                return e  # Or whatever message you want.

        return wrapper

    return error_log

In [77]:
@log_error()
def divide(num1, num2):
    return num1 / num2


if __name__ == '__main__':
    result = divide(10, 0)
    print(result)

ERROR:LogError:And error has occurred at /divide
Traceback (most recent call last):
  File "/tmp/ipykernel_14562/1587134262.py", line 44, in wrapper
    return func(*args, **kwargs)
  File "/tmp/ipykernel_14562/2516426464.py", line 3, in divide
    return num1 / num2
ZeroDivisionError: division by zero


division by zero


#  <a name="conclusao"> Conclusão </a>
[Voltar ao índice](#indice)

Neste notebook vimos que é importante ter um estrutura no notebook para facilitar o desenvolvimento e divulgação do conhecimento